<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Notebook for Peer Assignment

Estimated time needed: 60 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Weather forecast firm as a data scientist.

The company is considering the weather condition to help predict the possibility of precipitations, which involves using various local climatological variables, including temperature, wind speed, humidity, dew point, and pressure. The data you will be handling was collected by a NOAA weather station located at the John F. Kennedy International Airport in Queens, New York.

Your task is to provide a high level analysis of weather data in JFK Airport. Your stakeholders want to understand the current and historical record of precipitations based on different variables. For now they are mainly interested in a macro-view of JFK Airport Weather, and how it relates to the possibility to rain because it will affect flight delays and etc.


# Introduction

This project relates to the NOAA Weather Dataset - JFK Airport (New York). The original dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature and wind speed) collected at JFK airport. This dataset can be obtained for free from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0151ENSkillsNetwork21582452-2022-01-01).

For this project, you will be using a subset dataset, which contains 5727 rows (about 5% or original rows) and 9 columns. The end goal will be to predict the precipitation using some of the available features. In this project, you will practice reading data files, preprocessing data, creating models, improving models and evaluating them to ultimately choose the best model.


## Table of Contents:

Using this R notebook you will complete **10 tasks**:

*   [0. Prerequisites](#cell0)
*   [1. Download and Unzip NOAA Weather Dataset](#cell1)
*   [2. Read Dataset into Project](#cell2)
*   [3. Select Subset of Columns](#cell3)
*   [4. Clean Up Columns](#cell4)
*   [5. Convert Columns to Numerical Types](#cell5)
*   [6. Rename Columns](#cell6)
*   [7. Exploratory Data Analysis](#cell7)
*   [8. Linear Regression](#cell8)
*   [9. Improve the Model](#cell9)
*   [10. Find Best Model](#cell10)

<a id="cell0"></a>

## 0. Prerequisites

**When creating this notebook in Watson Studio, remember to chose R under "Select runtime" to make sure the kernel is using R.**

Before you run this notebook complete the following steps:

*   Insert a project token
*   Import required modules

#### Insert a project token

When you import this project from the Watson Studio Gallery, a token should be automatically generated and inserted at the top of this notebook as a code cell such as the one below:

```python
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='YOUR_PROJECT_ID', project_access_token='YOUR_PROJECT_TOKEN')
pc = project.project_context
```

If you do not see the cell above, follow these steps to enable the notebook to access the dataset from the project's resources:

*   Click on `More -> Insert project token` in the top-right menu section

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

*   This should insert a cell at the top of this notebook similar to the example given above.

    > If an error is displayed indicating that no project token is defined, follow [these instructions](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/token.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0151ENSkillsNetwork21582452-2022-01-01&audience=wdp&context=data).

*   Run the newly inserted cell before proceeding with the notebook execution below

### Import required modules

Below, install "tidymodels", additionally "rlang" should be updated in order to properly run "tidymodels".


In [3]:
# Install tidymodels if you haven't done so
install.packages("rlang")
install.packages("tidymodels")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


After installing the packages, load them. Tidyverse and Tidymodels will be the two main packages you will use.


In [4]:
# Library for modeling
library(tidymodels)

# Load tidyverse
library(tidyverse)

── Attaching packages ────────────────────────────────────── tidymodels 0.2.0 ──
✔ broom        0.8.0     ✔ recipes      0.2.0
✔ dials        0.1.1     ✔ rsample      0.1.1
✔ dplyr        1.0.9     ✔ tibble       3.1.7
✔ ggplot2      3.3.6     ✔ tidyr        1.2.0
✔ infer        1.0.0     ✔ tune         0.2.0
✔ modeldata    0.1.1     ✔ workflows    0.2.6
✔ parsnip      0.2.1     ✔ workflowsets 0.2.1
✔ purrr        0.3.4     ✔ yardstick    0.0.9
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ readr   1.3.1     ✔ forcats 0.5.0
✔ stringr 1.4.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks

### Understand the Dataset

The original NOAA JFK dataset contains 114,546 hourly observations of various local climatological variables (including temperature, wind speed, humidity, dew point, and pressure).

In this project you will use a sample dataset, which is around 293 KB. [Link to the sample dataset](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz).

The sample contains 5727 rows (about 5% or original rows) and 9 columns, which are:

*   DATE
*   HOURLYDewPointTempF
*   HOURLYRelativeHumidity
*   HOURLYDRYBULBTEMPF
*   HOURLYWETBULBTEMPF
*   HOURLYPrecip
*   HOURLYWindSpeed
*   HOURLYSeaLevelPressure
*   HOURLYStationPressure

The original dataset is much bigger. Feel free to explore the original dataset. [Link to the original dataset.](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa_weather.html)

For more information about the dataset, checkout the [preview](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/data-preview/index.html?\_ga=2.176781478.281508226.1616293518-1509963377.1616117067&cm_mc_uid=90945889198916153255549&cm_mc_sid\_50200000=64650651616293516933) of NOAA Weather - JFK Airport.


<a id="cell1"></a>

## 1. Download NOAA Weather Dataset

Use the `download.file()` function to download the sample dataset from the URL below.

URL = '<https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz>'


In [5]:
url <- "https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz"
download.file(url,destfile = "noaa-weather-sample-data.tar.gz")

Untar the zipped file.


In [6]:
untar("noaa-weather-sample-data.tar.gz", tar = "internal")

Warning message in untar2(tarfile, files, list, exdir, restore_times):
“using pax extended headers”

<a id="cell2"></a>

## 2. Extract and Read into Project

We start by reading in the raw dataset. You should specify the file name as "noaa-weather-sample-data/jfk_weather_sample.csv".


In [7]:
noaa_weather_data <- read_csv("noaa-weather-sample-data/jfk_weather_sample.csv",
                     col_types = cols('HOURLYDewPointTempF' = col_number(),
                                     'HOURLYWETBULBTEMPF' = col_number(),
                                         'HOURLYPrecip'= col_number(),
                                       'HOURLYWindSpeed'= col_number(),
                                  'HOURLYSeaLevelPressure'= col_number(),
                                  'HOURLYStationPressure'= col_number()))

Warning message:
“392 parsing failures.
row          col expected actual                                              file
  5 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 29 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 41 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 49 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 64 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
... ............ ........ ...... .................................................
See problems(...) for more details.
”

Next, display the first few rows of the dataframe.


In [8]:
head (noaa_weather_data)

DATE,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYPrecip,HOURLYWindSpeed,HOURLYSeaLevelPressure,HOURLYStationPressure
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015-07-25 13:51:00,60,46,83,68,0,13,30.01,29.99
2016-11-18 23:51:00,34,48,53,44,0,6,30.05,30.03
2013-01-06 08:51:00,33,89,36,35,0,13,30.14,30.12
2011-01-27 16:51:00,18,48,36,30,0,14,29.82,29.80
2015-01-03 12:16:00,27,61,39,34,NA,11,NA,30.50
2013-02-15 20:51:00,35,79,41,38,0,6,29.94,29.92


Also, take a `glimpse` of the dataset to see the different column data types and make sure it is the correct subset dataset with about 5700 rows and 9 columns.


In [9]:
glimpse(noaa_weather_data)

Rows: 5,727
Columns: 9
$ DATE                   <dttm> 2015-07-25 13:51:00, 2016-11-18 23:51:00, 2013…
$ HOURLYDewPointTempF    <dbl> 60, 34, 33, 18, 27, 35, 4, 14, 51, 71, 76, 19, …
$ HOURLYRelativeHumidity <dbl> 46, 48, 89, 48, 61, 79, 51, 65, 90, 94, 79, 37,…
$ HOURLYDRYBULBTEMPF     <dbl> 83, 53, 36, 36, 39, 41, 19, 24, 54, 73, 83, 44,…
$ HOURLYWETBULBTEMPF     <dbl> 68, 44, 35, 30, 34, 38, 15, 21, 52, 72, 78, 35,…
$ HOURLYPrecip           <dbl> 0.00, 0.00, 0.00, 0.00, NA, 0.00, 0.00, 0.00, 0…
$ HOURLYWindSpeed        <dbl> 13, 6, 13, 14, 11, 6, 0, 11, 11, 5, 21, 7, 17, …
$ HOURLYSeaLevelPressure <dbl> 30.01, 30.05, 30.14, 29.82, NA, 29.94, 30.42, 3…
$ HOURLYStationPressure  <dbl> 29.99, 30.03, 30.12, 29.80, 30.50, 29.92, 30.40…


<a id="cell3"></a>

## 3. Select Subset of Columns

The end goal of this project will be to predict `HOURLYprecip` (precipitation) using a few other variables. Before you can do this, you first need to preprocess the dataset. Section 3 to section 6 focuses on preprocessing.

The first step in preprocessing is to select a subset of data columns and inspect the column types.

The key columns that we will explore in this project are:

*   HOURLYRelativeHumidity
*   HOURLYDRYBULBTEMPF
*   HOURLYPrecip
*   HOURLYWindSpeed
*   HOURLYStationPressure

Data Glossary:

*   'HOURLYRelativeHumidity' is the relative humidity given to the nearest whole percentage.
*   'HOURLYDRYBULBTEMPF' is the dry-bulb temperature and is commonly used as the standard air temperature reported. It is given here in whole degrees Fahrenheit.
*   'HOURLYPrecip' is the amount of precipitation in inches to hundredths over the past hour. For certain automated stations, precipitation will be reported at sub-hourly intervals (e.g. every 15 or 20 minutes) as an accumulated amount of all precipitation within the preceding hour. A “T” indicates a trace amount of precipitation.
*   'HOURLYWindSpeed' is the speed of the wind at the time of observation given in miles per hour (mph).
*   'HOURLYStationPressure' is the atmospheric pressure observed at the station during the time of observation. Given in inches of Mercury (in Hg).

`Select` those five columns and store the modified dataframe as a new variable.


In [10]:
weather_data_sub <- noaa_weather_data %>% 
    select(HOURLYRelativeHumidity,
                HOURLYDRYBULBTEMPF,
                HOURLYPrecip,
                HOURLYWindSpeed,
                HOURLYStationPressure)

Show the first 10 rows of this new dataframe.


In [11]:
head(weather_data_sub, 10)

HOURLYRelativeHumidity,HOURLYDRYBULBTEMPF,HOURLYPrecip,HOURLYWindSpeed,HOURLYStationPressure
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
46,83,0.00,13,29.99
48,53,0.00,6,30.03
89,36,0.00,13,30.12
48,36,0.00,14,29.80
61,39,NA,11,30.50
79,41,0.00,6,29.92
51,19,0.00,0,30.40
65,24,0.00,11,30.35
90,54,0.06,11,30.03


<a id="cell4"></a>

## 4. Clean Up Columns

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NA` and `T` values. `T` specifies *trace amounts of precipitation* (meaning essentially no precipitation), while `NA` means *not available*, and is used to denote missing values. Additionally, some values also have "s" at the end of them, indicating that the precipitation was snow.

Inspect the unique values present in the column `HOURLYPrecip` (with `unique(dataframe$column)`) to see these values.


In [12]:
unique(weather_data_sub$HOURLYPrecip)

[1] 0.00   NA 0.06 0.03 0.02 0.08 0.01 0.07 0.16 0.09 0.22 0.24 0.18 0.05 0.04
[16] 0.11 0.14 0.25 0.10 0.58 0.12 0.13 0.46 1.07 1.19 0.34 0.20 0.36 0.42 0.17
[31] 0.27 0.35 0.31 0.33 0.23 0.26 0.28 0.75 0.19 0.54 0.59 0.21

Having characters in values (like the "T" and "s" that you see in the unique values) will cause problems when you create a model because values for precipitation should be numerical. So you need to fix these values that have characters.

Now, for the column `HOURLYPrecip`:

1.  Replace all the `T` values with "0.0" and
2.  Remove "s" from values like "0.02s". In R, you can use the method `str_remove(column, pattern = "s$")` to remove the character "s" from the end of values. The "$" tells R to match to the end of values. The `pattern` is a regex pattern. Look at [here](https://www.rdocumentation.org/packages/stringi/versions/1.5.3/topics/about_search_regex?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0151ENSkillsNetwork21582452-2022-01-01) for more information about regex and matching to strings in R.

Remember that you can use `tidyverse`'s  `mutate()` to update columns.

You can check your work by checking if unique values of `HOURLYPrecip` still contain any `T` or `s`. Store the modified dataframe as a new variable.


In [13]:
weather_data_sub$HOURLYPrecip[weather_data_sub$HOURLYPrecip=="T"]<-"0.0"
weather_data_sub

HOURLYRelativeHumidity,HOURLYDRYBULBTEMPF,HOURLYPrecip,HOURLYWindSpeed,HOURLYStationPressure
<dbl>,<dbl>,<chr>,<dbl>,<dbl>
46,83,0,13,29.99
48,53,0,6,30.03
89,36,0,13,30.12
48,36,0,14,29.80
61,39,NA,11,30.50
79,41,0,6,29.92
51,19,0,0,30.40
65,24,0,11,30.35
90,54,0.06,11,30.03


In [14]:
str_remove(weather_data_sub$HOURLYPrecip, pattern = "s$")

[1] "0"    "0"    "0"    "0"    NA     "0"    "0"    "0"    "0.06" NA    
  [11] NA     "0"    "0"    "0"    "0"    "0"    "0"    NA     "0.03" NA    
  [21] "0.03" NA     "0"    "0"    "0.02" NA     "0"    NA     NA     "0"   
  [31] "0"    NA     "0"    "0"    "0"    NA     "0"    "0"    "0.08" NA    
  [41] NA     NA     NA     "0"    "0"    "0"    "0.01" "0"    NA     NA    
  [51] "0"    "0"    NA     "0"    "0"    NA     "0"    "0"    "0"    "0"   
  [61] "0"    NA     NA     NA     "0"    NA     "0"    "0"    NA     "0"   
  [71] "0"    "0"    "0.07" NA     "0"    NA     "0"    "0"    NA     NA    
  [81] "0"    NA     NA     "0"    "0"    NA     "0"    NA     NA     "0"   
  [91] NA     NA     "0"    "0"    NA     "0"    "0.16" "0"    "0"    "0"   
 [101] "0"    "0.08" "0"    "0"    "0"    NA     "0"    "0"    NA     NA    
 [111] NA     NA     "0"    NA     NA     "0"    "0"    "0"    "0.09" "0.06"
 [121] NA     "0"    "0"    NA     NA     "0"    "0"    NA     NA     "0"   
 [131] "0"    "0"    "0"    "0"    NA     "0"    "0"    "0"    "0"    "0"   
 [141] NA     "0"    NA     "0"    NA     "0.22" "0"    NA     "0"    NA    
 [151] "0.02" NA     NA     "0"    "0"    "0"    NA     "0"    NA     NA    
 [161] NA     NA     NA     NA     NA     "0.02" "0"    "0"    "0"    NA    
 [171] NA     "0"    "0"    NA     "0"    "0"    "0"    NA     "0.01" "0"   
 [181] "0"    NA     NA     "0"    NA     NA     "0"    NA     "0"    "0"   
 [191] "0"    "0"    "0.24" NA     "0"    "0.03" NA     "0"    NA     "0"   
 [201] "0"    "0"    "0"    "0"    NA     "0"    NA     "0"    "0"    "0"   
 [211] NA     NA     "0"    NA     NA     "0"    "0"    "0"    NA     "0"   
 [221] "0"    "0"    "0"    "0"    "0"    NA     "0"    "0"    "0"    "0.03"
 [231] "0"    NA     NA     "0"    "0"    NA     "0"    NA     "0"    NA    
 [241] NA     NA     "0"    "0"    "0"    "0"    NA     "0"    "0"    NA    
 [251] NA     NA     "0"    "0"    "0.01" NA     "0"    NA     "0"    "0"   
 [261] "0"    NA     "0"    NA     NA     "0"    NA     "0"    NA     "0"   
 [271] "0"    "0"    NA     "0"    "0"    NA     NA     "0"    NA     NA    
 [281] "0.18" "0"    "0"    "0"    NA     "0"    "0"    NA     "0"    "0.03"
 [291] "0"    NA     NA     NA     "0"    "0"    NA     NA     "0.05" "0"   
 [301] "0.03" "0"    "0"    "0"    "0"    "0"    "0"    NA     NA     "0"   
 [311] NA     "0"    NA     "0"    "0"    "0"    "0"    "0"    "0"    NA    
 [321] NA     "0"    "0"    NA     "0"    "0"    "0"    NA     "0"    NA    
 [331] NA     NA     NA     "0"    "0"    "0"    "0.06" "0"    "0"    NA    
 [341] NA     NA     "0"    "0"    "0"    "0"    NA     "0"    "0"    NA    
 [351] NA     NA     NA     "0"    NA     "0"    "0"    "0.01" "0"    NA    
 [361] "0.06" "0"    NA     "0"    "0"    "0.04" "0.09" "0"    "0"    "0"   
 [371] "0"    "0"    "0"    NA     NA     NA     NA     "0"    NA     "0"   
 [381] "0"    NA     NA     "0"    NA     "0"    "0"    "0"    "0"    NA    
 [391] NA     NA     "0"    NA     "0"    "0"    "0"    "0"    "0"    "0"   
 [401] NA     "0"    "0"    "0.07" NA     NA     "0.11" "0"    "0"    NA    
 [411] NA     "0"    "0"    "0"    "0"    "0"    NA     NA     "0"    NA    
 [421] "0"    "0"    "0"    "0"    "0"    "0"    "0"    "0"    NA     "0"   
 [431] "0"    NA     "0"    "0"    NA     "0"    NA     "0"    "0"    "0"   
 [441] "0"    NA     "0"    NA     NA     "0"    "0"    "0"    NA     NA    
 [451] NA     "0"    "0"    NA     NA     "0"    "0"    NA     "0"    "0"   
 [461] NA     NA     "0"    "0.09" NA     "0"    "0"    NA     NA     NA    
 [471] "0.06" NA     "0"    NA     "0"    NA     "0"    "0"    "0"    "0.01"
 [481] NA     "0"    "0.01" "0"    "0"    "0"    "0"    "0"    "0"    "0"   
 [491] "0"    NA     "0"    "0"    NA     NA     "0"    "0"    "0"    "0"   
 [501] NA     "0"    "0"    "0"    NA     NA     NA     NA     "0.01" "0"   
 [511] "0"    "0"    NA     "0"    "0"    "0"    NA     NA     "0"    "0"   

In [16]:
map(weather_data_sub, ~sum(is.na(.)))

$HOURLYRelativeHumidity
[1] 151

$HOURLYDRYBULBTEMPF
[1] 151

$HOURLYPrecip
[1] 2052

$HOURLYWindSpeed
[1] 156

$HOURLYStationPressure
[1] 158

In [17]:
dim(weather_data_sub)

[1] 5727    5

<a id="cell5"></a>

## 5. Convert Columns to Numerical Types

Now that you have removed the characters in the `HOURLYPrecip` column, you can safely covert the column to a numeric type.

First, check the types of the columns. You will notice that all are `dbl` (double or numeric) except for `HOURLYPrecip`, which is `chr` (character or string). Use the `glimpse` function from Tidyverse.


In [21]:
weather_data_sub %>% 
    summarize_all(class) %>% 
    gather(variable, class)

variable,class
<chr>,<chr>
HOURLYRelativeHumidity,numeric
HOURLYDRYBULBTEMPF,numeric
HOURLYPrecip,character
HOURLYWindSpeed,numeric
HOURLYStationPressure,numeric


Convert `HOURLYPrecip` to the `numeric` type and store the cleaned dataframe as a new variable.


In [24]:
weather_data_sub %>%
    select(HOURLYPrecip) %>%
    mutate_all(type.convert) %>%
    mutate_if(is.character, as.numeric)

HOURLYPrecip
<dbl>
0.00
0.00
0.00
0.00
NA
0.00
0.00
0.00
0.06


We can now see that all fields have numerical data type.


In [26]:
weather_data_sub %>% 
    summarize_all(class) %>% 
    gather(variable, class)

variable,class
<chr>,<chr>
HOURLYRelativeHumidity,numeric
HOURLYDRYBULBTEMPF,numeric
HOURLYPrecip,character
HOURLYWindSpeed,numeric
HOURLYStationPressure,numeric


<a id="cell6"></a>

## 6. Rename Columns

Let's rename the following columns as:

*   'HOURLYRelativeHumidity' to 'relative_humidity'
*   'HOURLYDRYBULBTEMPF' to 'dry_bulb_temp_f'
*   'HOURLYPrecip' to 'precip'
*   'HOURLYWindSpeed' to 'wind_speed'
*   'HOURLYStationPressure' to 'station_pressure'

You can use `dplyr::rename()`. Then, store the final dataframe as a new variable.


In [34]:
library(dplyr)
weather_data_sub %>% 
    rename_all (relative_humidity = HOURLYRelativeHumidity,
              dry_bulb_temp = HOURLYDRYBULBTEMPF,
              precip = HOURLYPrecip,
              wind_speed = HOURLYWindSpeed,
              station_pressure = HOURLYStationPressure)

ERROR: Error in list2(...): object 'HOURLYRelativeHumidity' not found


<a id="cell7"></a>

## 7. Exploratory Data Analysis

Now that you have finished preprocessing the dataset, you can can start exploring the columns more.

First, split the data into a training and testing set. Splitting a dataset is done randomly, so to have reproducible results set the seed = 1234. Also, use 80% of the data for training.


Next, looking at just the **training set**, plot histograms or box plots of the variables (`relative_humidity`, `dry_bulb_temp_f`, `precip`, `wind_speed`,  `station_pressure`) for an intial look of their distributions using `tidyverse`'s `ggplot`. Leave the testing set as is because it is good practice to not see the testing set until evaluating the final model.


<a id="cell8"></a>

## 8. Linear Regression

After exploring the dataset more, you are now ready to start creating models to predict the precipitation (`precip`).

Create simple linear regression models where `precip` is the response variable and each of `relative_humidity`, `dry_bulb_temp_f`,`wind_speed` or `station_pressure` will be a predictor variable, e.g. `precip ~ relative_humidity`, `precip ~ dry_bulb_temp_f`, etc. for a total of four simple models.
Additionally, visualize each simple model with a scatter plot.


<a id="cell9"></a>

## 9. Improve the Model

Now, try improving the simple models you created in the previous section.

Create at least two more models, each model should use at least one of the different techniques:

1.  Add more features/predictors
2.  Add regularization (L1, L2 or a mix)
3.  Add a polynomial component

Also, for each of the models you create, check the model performance using the **training set** and a metric like MSE, RMSE, or R-squared.

Consider using `tidymodels` if you choose to add regularization and tune lambda.


<a id="cell10"></a>

## 10. Find Best Model

Compare the regression metrics of each model from section 9 to find the best model overall. To do this,

1.  Evaluate the models on the **testing set** using at least one metric (like MSE, RMSE or R-squared).
2.  After calculating the metrics on the testing set for each model, print them out in as a table to easily compare. You can use something like:

```
model_names <- c("model_1", "model_2", "model_3")
train_error <- c("model_1_value", "model_2_value", "model_3_value")
test_error <- c("model_1_value", "model_2_value", "model_3_value")
comparison_df <- data.frame(model_names, train_error, test_error)
```

3.  Finally, from the comparison table you create, conclude which model performed the best.


## Author(s)

<h4> Yiwen Li </h4>

## Contributions

<h4> Tiffany Zhu </h4>

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
